<a href="https://colab.research.google.com/github/filipkowskii/MEB/blob/master/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
#brings in Dotmap

import sys
sys.path.insert(1, "/content/CHE2064") 
#Path to CHE module imports

In [2]:
from dotmap import DotMap
import pandas as pd
import jax
from jax.lax import stop_gradient
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision
from tools.tree_array_transform import VSC, Comp, Range
import tools.che as che
R=8.314 # J/(mol K)

In [3]:
p = che.Props(['Nitrogen','Oxygen', 'Argon', 'Water'])
#compound information given by Co's program

In [4]:
def model(c,r):
    # c: combination of adjustable variables and static state parameters
    # r: DotMap - store intermediate results for reporting

    r.Pw = p.Pvap(c.T)[3] 
    # gives partial pressure of water- WOULD be found using Antoine's equation, but Co stores it for us
    # [3] = water information - index 3 of che.Props

    r.P = c.air_n * R * c.T / c.V_vap + r.Pw
    #first unknown equation ( P = (nair*R*T)/Vvap) + Pvap(T) )

    return (c.P_f, r.P)

c=DotMap()
#stores values in a dotmap

c.W_tot = 1. # 1 kg
c.V_tot = 0.01 # 10 Liters
c.P_i = 1e5 # Pa air pressure
c.P_f = 2e5 # Pa final pressure
c.T_i = 298.
# all above are values given in the problem

c.Vw_i = c.W_tot/p.rhol(c.T_i)[3]
# initial volume of water = mass / density of water

c.V_vap = c.V_tot - c.Vw_i 
# volume of air (vapor phase) = total volume - initial volume of water (given)
# Approximation - water in the vapor phase is negligible

c.air_n = c.P_i * c.V_vap / (R * c.T_i)
# moles of air = initial pressure * volume of air (vapor phase)/(R * initial temperature)

c.T = Range(350,300,600)
# range for temp. values (reasonable guess- 350, min-300, max-600)

/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [5]:
# VSC is code written by Co as well to transform our model into one that takes a single 
# array of adjustable variables for minimization.  
# Unnecessary for this simple model with just 1 variable, 
# but very helpful for more complex problems later.

vsc=VSC(c,model)
vsc.solve()

# now we've set the parameter of our desired pressure

0.001973574785335787
0.0006606517658790154
0.0006606517658790154
1.5272366214780464e-05
1.5272366214780464e-05
1.5272366214780464e-05
1.213645372823828e-08
1.213645372823828e-08
1.213645372823828e-08
6.020159869574761e-14
6.020159869574761e-14
6.020159869574761e-14
6.020159869574761e-14
6.020159869574761e-14
6.409037068602645e-20


In [6]:
# DataFrame of adjustable variables -- (vdf == variable data frame) temperature!
vsc.vdf

,vector1
,1
T,365.719235


In [7]:
# Dataframe of intermediate results (stuff up top beginning with r.____)
vsc.rdf

,vector1
,1
Pw,77275.424480
P,200000.000101


In [8]:
# Dataframe of static state parameters (sdf == static data frame) -- shows everything we started with inputting
vsc.sdf

,vector1
,1
Vw_i,0.001005
P_i,100000.000000
air_n,0.363057
T_i,298.000000
V_vap,0.008995
P_f,200000.000000
V_tot,0.010000
W_tot,1.000000


In [11]:
# on to setting the paramter of our desired amount of moles of water vapor
# copy and paste from original model and change from c. to r. because these are now
# intermediate values, total volume (c.V_tot) is now our adjustable parameter

def model2(c,r):
    r.Pw = p.Pvap(c.T)[3]
    r.V_vap = c.V_tot - c.Vw_i # Approximation - water in the vapor phase is negligible
    r.air_n = c.P_i * r.V_vap / (R * c.T_i)
    # moles of air

    r.W_n_vap = r.Pw * r.V_vap / (R * c.T) 
    # moles of water in the vapor phase -- 
    # partial pressure of water * volume of vapor phase / RT

    r.P = r.air_n  * R * c.T / r.V_vap + r.Pw

    P_constraint = (c.P_f, r.P)
    # pressure constraint -- makes so that the pressure we calculate is equal to
    # the desired final pressure

    W_n_constraint = (r.W_n_vap, c.W_n_vap_desired)
    # number moles in vapor phase constraint -- makes so that calculated moles of
    # water in the vapor phase is equal to the desired amount (0.3)

    return (P_constraint, W_n_constraint)

In [17]:
c=DotMap()
c.W_tot = 1. # 1 kg
c.V_tot = 0.01 # 10 Liters
c.P_i = 1e5 # Pa air pressure
c.P_f = 2e5 # Pa final pressure
c.T_i = 298.

c.W_n_vap_desired = 0.3 
# want 0.3 moles of water in the vapor phase

c.Vw_i = c.W_tot/p.rhol(c.T_i)[3]
# initial volume of water

c.T = Range(350, 300, 400) 
# final temperature

c.V_tot = Range(0.01, 0., 0.3) 
# total container volume -- made adjustable so that we can solve for desired number
# of moles of water in vapor phase (initial guess, min, max)

vsc=VSC(c,model2)
vsc.solve()

0.17209294672665715
0.008306620035433327
0.008306620035433327
0.0015259971137814683
0.00014629191998891317
0.00014629191998891317
3.665611594208479e-07
3.665611594208479e-07
3.665611594208479e-07
6.630472553863636e-12
6.630472553863636e-12
6.630472553863636e-12
6.630472553863636e-12
6.630472553863636e-12
5.229802711082758e-17
5.229802711082758e-17
5.229802711082758e-17
5.229802711082758e-17
3.3766925003181233e-21


In [18]:
vsc.vdf
# shows now that to have 0.3 moles of water in the vapor phase, our temperature
# stays roughly the same, but our container needs to be about 12 liters - before
# it was 10 liters

,vector1
,1
T,365.719235
V_tot,0.012809


In [19]:
vsc.rdf
# shows our intermediate values again -- shows that desired pressure
# is still at 200000 Pa

,vector1
,1
Pw,77275.424370
V_vap,0.011804
air_n,0.476443
W_n_vap,0.300000
P,199999.999978
